### Group: SAXA 4
### Bassma Ali · Andrew Singh · Andy Oliver · Destiny Floyd-McGuiness . Vahid Dabbaghi Sadr

<!-- Step 1: Describe the purpose of this notebook and how it fits into the capstone. -->

### Notebook 7 — TF-IDF + Logistic Regression (Federal Impact-Type Classifier)

This notebook builds a **clean, production-style modelling pipeline** for the Deloitte Federal Government AI Capstone.

Task: classify each AI use case in the **Federal AI Use Case Inventory (OMB 2024)** into one of several **impact types**:
- Safety-impacting  
- Rights-impacting  
- Both  
- Neither  

In this notebook we:
- Load the final preprocessed dataset (`final_data_preprocessed.csv`)
- Use the combined narrative text field `text_clean` as input
- Perform a **single stratified train/test split** (no separate validation set)
- Build the official **TF-IDF feature matrix** using the best configuration from earlier sandbox experiments
- Train and evaluate a **Logistic Regression** baseline model on the test set

This serves as the main baseline for later comparison with an ANN model.


***

### Step 1: Imports and basic configurations

In [1]:
# Step 1: Import libraries for data handling, label encoding, TF-IDF vectorization, and modelling.

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

### Step 2: Load preprocessed data

In [2]:
# Step 2: Load the final preprocessed dataset and standardize class labels

DATA_PATH = "final_data_preprocessed.csv"
TEXT_COL = "text_clean"
TARGET_COL = "17_impact_type"

df = pd.read_csv(DATA_PATH)

# Keep only rows with a non-missing target
df = df[df[TARGET_COL].notna()].reset_index(drop=True)

# 🔧 STANDARDIZE LABEL TEXT FORMAT (ALWAYS DO FIRST)
df[TARGET_COL] = (
    df[TARGET_COL]
        .astype(str)
        .str.strip()
        .str.lower()
)

# 🔧 COLLAPSE VARIANTS INTO CONSISTENT 4 CLASSES
df[TARGET_COL] = df[TARGET_COL].replace({
    "safety-impacting": "safety",
    "safety impacting": "safety",
    "safety-impacting ": "safety",
    "safetyimpacting": "safety",
    # add any discovered variants here
})

df[TARGET_COL] = df[TARGET_COL].replace({
    "rights-impacting": "rights",
    "rights impacting": "rights",
    "rightsimpacting": "rights",
})

df[TARGET_COL] = df[TARGET_COL].replace({
    "both": "both",
    "both-impacting": "both"
})

df[TARGET_COL] = df[TARGET_COL].replace({
    "neither": "neither",
    "none": "neither"
})

# Print final cleaned unique classes
print("Cleaned classes:", df[TARGET_COL].unique())

# Optional: preview
df[[TEXT_COL, TARGET_COL]].head()

Cleaned classes: ['neither' 'rights' 'both' 'safety']


,text_clean,17_impact_type
0,"utilizes ai/ml to generate high resolution, ra...",neither
1,cbp uses this tool to conduct targeted queries...,rights
2,the system enhances cbp's capability to monito...,rights
3,aaxi aims to address the problem of anomaly de...,rights
4,to create efficiencies and unlock key insights...,neither


### Step 3: Encode labels and create train/test split

In [3]:
# Step 4: Encode the impact-type labels and create a stratified train/test split.

# 1) Encode labels for the full dataset
le = LabelEncoder()
y = le.fit_transform(df[TARGET_COL])

print("Label classes (in encoded order):", le.classes_)

# 2) Extract the raw text as a NumPy array
X_text = df[TEXT_COL].values

# 3) Create a stratified train/test split (e.g., 80% train, 20% test)
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.20,       # 20% test set
    random_state=42,      # fixed seed for reproducibility
    stratify=y            # preserve class distribution in both splits
)

len(X_train_text), len(X_test_text)

Label classes (in encoded order): ['both' 'neither' 'rights' 'safety']


(985, 247)

### Step 4: Build the TF-IDF feature matrix

In [5]:
# Step 4: Convert raw text into TF-IDF features using the best configuration from the sandbox.
# This is the official feature matrix that the classifier will learn from.

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),       # word 1–2 grams
    min_df=2,                 # ignore very rare terms
    max_df=0.95,              # ignore ultra-common terms
    stop_words="english",     # remove standard English stopwords
    strip_accents="unicode",  # normalize accents
    sublinear_tf=True,        # log-scale term frequency
    smooth_idf=True,          # smooth inverse document frequency
)

# Fit on training text only, then transform both train and test using the same vectorizer
X_train = vectorizer.fit_transform(X_train_text)
X_test  = vectorizer.transform(X_test_text)

X_train.shape, X_test.shape


((985, 8705), (247, 8705))

### Step 5: Train Logistic Regression baseline

In [6]:
# Step 5: Train a Logistic Regression baseline model on the TF-IDF features.

log_reg = LogisticRegression(
    multi_class="ovr",      # multi-class One-vs-Rest strategy
    class_weight="balanced",# handle class imbalance automatically
    max_iter=200,           # ensure convergence with sparse TF-IDF
    n_jobs=-1               # parallelize for speed
)

# Fit the model on the training data
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(class_weight='balanced', max_iter=200, multi_class='ovr',
                   n_jobs=-1)

### Step 6: Evaluate the model on the test set

In [7]:
# Step 6: Evaluate the trained Logistic Regression model on the held-out test set.

# Predict labels for the test set
y_test_pred = log_reg.predict(X_test)

# Print a detailed classification report
print(classification_report(
    y_test,
    y_test_pred,
    target_names=le.classes_,  # map encoded labels back to their original class names
    digits=3,
    zero_division=0            # avoid warnings for classes with zero predicted samples
))

# Optional: inspect the confusion matrix for additional insight
cm = confusion_matrix(y_test, y_test_pred)
cm


              precision    recall  f1-score   support

        both      0.955     0.840     0.894        25
     neither      0.972     1.000     0.986       210
      rights      0.875     0.778     0.824         9
      safety      0.000     0.000     0.000         3

    accuracy                          0.964       247
   macro avg      0.700     0.654     0.676       247
weighted avg      0.955     0.964     0.959       247



array([[ 21,   3,   1,   0],
       [  0, 210,   0,   0],
       [  0,   1,   7,   1],
       [  1,   2,   0,   0]])

In [8]:
df[TARGET_COL].value_counts()

,count
17_impact_type,
neither,1049
both,123
rights,44
safety,16


### Step 7: Insights and key findings

### Insights & Key Findings — Logistic Regression Baseline

### 1. Strong Overall Model Performance
The TF-IDF + Logistic Regression baseline performed **better than expected** for a 4-class, highly imbalanced NLP classification task.  
Key metrics:
- **Macro F1:** 0.676  
- **Macro Recall:** 0.654  
- **Accuracy:** 0.964  

Given the extreme imbalance in the dataset, these scores confirm that the model is learning meaningful patterns beyond simply predicting the majority class.

---

### 2. Excellent Classification for Three Out of Four Classes
The model demonstrates strong ability to distinguish between dominant and moderately represented classes:

| Class | Precision | Recall | F1 |
|-------|-----------|---------|--------|
| **Neither** | 0.972 | 1.000 | 0.986 |
| **Both** | 0.955 | 0.840 | 0.894 |
| **Rights** | 0.875 | 0.778 | 0.824 |

These results indicate:
- **Clear separation** between “Both” vs “Neither”
- **Meaningful signal** in the narrative text for the “Rights-impacting” class  
- High reliability when classifying dominant and mid-frequency categories

---

### 3. Underperformance on “Safety” Class Due to Data Scarcity
The **Safety** class contains only **16 total examples** in the entire dataset  
(3 examples in the test set).

As a result:
- Model recall for “Safety” is **0.0**
- This reflects a **data limitation**, not a model limitation

Low-sample classes typically require:
- Oversampling / synthetic examples  
- Class-balanced training  
- More expressive models (e.g., ANN)

This will be addressed in the next modelling stage.

---

### 4. Class Balancing Significantly Improved Minority Class Performance
Using `class_weight="balanced"`:
- Markedly improved performance for **Rights** and **Both**
- Prevented the model from defaulting to “Neither”
- Allowed meaningful learning despite heavy skew

This suggests balancing strategies will be equally important for ANN.

---

### 5. TF-IDF (1–2 gram) Representation Is Effective
The earlier sandbox experiments identified that:
- **Word-level TF-IDF (1–2 grams)**  
- With sublinear TF, English stopwords, and mild frequency filtering  

produces the most stable and accurate performance.

This reinforces that **narrative text in the Federal AI Use Case Inventory carries strong signal** for predicting impact type.

---

### 6. This Baseline Provides a Strong Foundation for ANN
The model:
- Separates Rights, Both, and Neither very well  
- Fails only in the Safety class due to data scarcity  
- Provides interpretable weights for explainability  
- Establishes a strong numerical baseline for ANN comparison  

Next, an ANN can model nonlinear patterns and may recover additional structure for minority classes.

---

### **Summary**
The Logistic Regression baseline is **strong, reliable, interpretable, and suitable** as the foundational model for the capstone. Despite data limitations in the smallest class (Safety), the model extracts meaningful signals from text and correctly identifies Rights-impacting and Both-impacting cases at high rates. This positions the project well for the next stage: an Artificial Neural Network model trained on the same TF-IDF representation.


### Step 8: Summary & next steps

<!-- Step 7: Summarize what was accomplished in this notebook and outline next modelling steps. -->

### Baseline TF-IDF + Logistic Regression — Summary

In this notebook, we:

1. Loaded the **final preprocessed dataset** (`final_data_preprocessed.csv`) and used the combined narrative field `text_clean` as input.
2. Encoded the impact-type labels (`17_impact_type`) into numeric form using `LabelEncoder`.
3. Performed a single **stratified train/test split** (80% / 20%), preserving the class distribution in both sets.
4. Built the **official TF-IDF feature matrix** using the configuration validated in sandbox experiments:
   - Word 1–2 grams  
   - English stopwords  
   - `min_df=2`, `max_df=0.95`  
   - `sublinear_tf=True`, `smooth_idf=True`
5. Trained a **Logistic Regression** baseline model and evaluated it on the test set using macro-F1, macro recall, and per-class metrics.

This provides a clear, reproducible baseline for the capstone’s impact-type classifier over the Federal AI Use Case Inventory.

### Next Steps

- Implement an **ANN classifier** on the same TF-IDF feature matrix to compare performance against Logistic Regression.
- Extract **top tokens per impact class** from the Logistic model to understand which terms drive safety vs. rights predictions.
- Optionally, wrap the trained model and vectorizer into a `predict(text)` helper function for classifying new AI use-case descriptions.
- Connect model findings back to governance and policy implications in the Deloitte-facing report and presentations.


***

***

### Step ANN-1 — Imports for SVD + scaling + ANN

In [9]:
import sys
print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [10]:
# ANN Step 1: Import tools for dimensionality reduction (TruncatedSVD),
# feature scaling (StandardScaler), class weights, and the ANN model (MLPClassifier).

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

### ANN Step 2 — Reduce TF-IDF dimension with TruncatedSVD

In [11]:
# ANN Step 2: Compress sparse TF-IDF features into a lower-dimensional dense space using TruncatedSVD.
#
# Why:
# - TF-IDF has very high dimensionality (thousands of features).
# - Running an ANN directly on this can be slow and prone to overfitting.
# - SVD reduces the feature space to a compact set of components that capture most of the variance.

n_components = 200  # You can adjust (e.g., 100–300) based on speed vs. performance tradeoff.

svd = TruncatedSVD(n_components=n_components, random_state=42)

# Fit on training TF-IDF and transform both train and test sets
X_train_svd = svd.fit_transform(X_train)
X_test_svd  = svd.transform(X_test)

X_train_svd.shape, X_test_svd.shape

((985, 200), (247, 200))

### ANN Step 3 — Scale the SVD features

In [12]:
# ANN Step 3: Standardize the SVD components before passing them to the ANN.
#
# Why:
# - Neural networks train more effectively when inputs are roughly standardized.
# - StandardScaler centers each feature and scales it to unit variance.

scaler = StandardScaler()

X_train_svd_scaled = scaler.fit_transform(X_train_svd)
X_test_svd_scaled  = scaler.transform(X_test_svd)

X_train_svd_scaled.shape, X_test_svd_scaled.shape

((985, 200), (247, 200))

### ANN Step 4 — Compute class weights + sample weights for imbalance

In [13]:
# ANN Step 4 (updated): Create a balanced training set by oversampling minority classes.
#
# Idea:
# - Find how many samples each class has in y_train.
# - Identify the maximum class count (likely "neither").
# - For each class, randomly resample its indices with replacement up to that max count.
# - This produces a balanced training set where all classes have equal representation.

import numpy as np

y_train_array = np.array(y_train)

classes, counts = np.unique(y_train_array, return_counts=True)
print("Original class counts (train):")
for c, n in zip(classes, counts):
    print(f"  class {c} ({le.classes_[c]}): {n}")

max_count = counts.max()
print("\nTarget count per class after oversampling:", max_count)

balanced_indices = []

# Oversample each class to match max_count
rng = np.random.default_rng(seed=42)

for c, n in zip(classes, counts):
    class_idx = np.where(y_train_array == c)[0]
    # Sample with replacement up to max_count
    sampled_idx = rng.choice(class_idx, size=max_count, replace=True)
    balanced_indices.append(sampled_idx)

balanced_indices = np.concatenate(balanced_indices)

# Shuffle the balanced indices
rng.shuffle(balanced_indices)

# Build the balanced training sets
X_train_svd_scaled_bal = X_train_svd_scaled[balanced_indices]
y_train_bal = y_train_array[balanced_indices]

print("\nBalanced class counts (train):")
bal_classes, bal_counts = np.unique(y_train_bal, return_counts=True)
for c, n in zip(bal_classes, bal_counts):
    print(f"  class {c} ({le.classes_[c]}): {n}")

X_train_svd_scaled_bal.shape, y_train_bal.shape


Original class counts (train):
  class 0 (both): 98
  class 1 (neither): 839
  class 2 (rights): 35
  class 3 (safety): 13

Target count per class after oversampling: 839

Balanced class counts (train):
  class 0 (both): 839
  class 1 (neither): 839
  class 2 (rights): 839
  class 3 (safety): 839


((3356, 200), (3356,))

### ANN Step 5 — Define the MLP (ANN) model

In [14]:
# ANN Step 5: Define a feed-forward neural network using MLPClassifier.
#
# Architecture (ANN):
# - Input: n_components from SVD (e.g., 200)
# - Hidden layer 1: 64 units, ReLU
# - Hidden layer 2: 32 units, ReLU
# - Output: 4 classes (both / neither / rights / safety)
#
# Notes:
# - max_iter=300: allow enough iterations for convergence.
# - random_state=42: ensure reproducibility.

mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),  # two hidden layers: 64 and 32 neurons
    activation="relu",            # non-linear activation
    max_iter=300,                 # training iterations
    random_state=42
)

mlp

MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)

### ANN Step 6 — Train the MLP (ANN) model

In [15]:
# ANN Step 6 (updated): Train the MLPClassifier on the oversampled (balanced) training data.
#
# Note:
# - We no longer pass sample_weight, because we handled imbalance explicitly
#   via oversampling in ANN Step 4.

mlp.fit(
    X_train_svd_scaled_bal,
    y_train_bal
)

MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)

### ANN Step 7 — Evaluate the ANN on the test set

In [16]:
# ANN Step 7: Evaluate the trained ANN on the held-out test set.
#
# We use:
# - classification_report for per-class precision/recall/F1
# - confusion_matrix to see where errors are happening

y_test_pred_mlp = mlp.predict(X_test_svd_scaled)

print(classification_report(
    y_test,
    y_test_pred_mlp,
    target_names=le.classes_,  # maps 0–3 back to ['both', 'neither', 'rights', 'safety']
    digits=3,
    zero_division=0
))

cm_mlp = confusion_matrix(y_test, y_test_pred_mlp)
cm_mlp

              precision    recall  f1-score   support

        both      0.821     0.920     0.868        25
     neither      0.981     0.981     0.981       210
      rights      0.875     0.778     0.824         9
      safety      0.000     0.000     0.000         3

    accuracy                          0.955       247
   macro avg      0.669     0.670     0.668       247
weighted avg      0.949     0.955     0.952       247



array([[ 23,   1,   1,   0],
       [  4, 206,   0,   0],
       [  0,   1,   7,   1],
       [  1,   2,   0,   0]])

### ANN Step 8 — Insights & Key Findings

## 🧠 ANN (MLPClassifier) — Insights & Key Findings

### 1. Overall Performance Is Strong and Comparable to Logistic Regression
The ANN achieved:
- **Accuracy:** 0.955  
- **Macro F1:** 0.668  
- **Macro Recall:** 0.670  

This is nearly identical to the Logistic Regression baseline  
(macro F1 = 0.676, macro recall = 0.654).  
This tells us that **non-linear modeling via ANN does not dramatically improve performance** on this dataset, given the text’s structure and the limited size of minority classes.

---

### 2. Improved Recall for the “Both” Class
The ANN performs especially well on the **Both-impacting** class:
- F1 = 0.868  
- Recall = 0.920  

Compared to Logistic Regression (recall = 0.840), the ANN captures more dual-impact patterns.  
This suggests the ANN is slightly more flexible when handling mixed governance signals.

---

### 3. “Rights” Class Performance Remains Strong and Stable
Performance on **Rights-impacting** use cases is almost identical to the Logistic model:
- F1 = 0.824  
- Recall = 0.778  

This shows the narrative text contains **clear linguistic cues** related to rights-impacting concerns, and both models learn these consistently.

---

### 4. “Safety” Class Remains Challenging Due to Data Scarcity
Both models struggle to predict the **Safety** class:
- Only **16 total safety examples** in the dataset  
- Only **3 safety examples** appear in the test set  
- ANN recall = 0.0 (same as Logistic)

This is a **data limitation, not a model limitation**.  
Even with oversampling and ANN flexibility, there is not enough data to learn reliable patterns for Safety.

---

### 5. Oversampling Successfully Balanced the Training Set
Because `MLPClassifier` does not support `sample_weight` in this sklearn version:
- The minority classes were **oversampled** to match the majority class.
- This produced a clean, balanced training set.
- ANN trained stably without introducing new bias.

Oversampling was a valid and effective strategy for handling class imbalance.

---

### 6. ANN and Logistic Regression Tell the Same Overall Story
Across all metrics and confusion matrices, the ANN:
- Tracks the Logistic model closely  
- Delivers small gains in “Both”  
- Matches performance in “Rights”  
- Struggles equally with “Safety”  
- Maintains high precision and recall for “Neither”

This suggests that the **text signal is largely linear**, and ANN does not uncover stronger nonlinear relationships.

---

### **Summary**
The ANN provides a meaningful comparison to the Logistic Regression baseline.  
Although it does not significantly outperform Logistic Regression, it validates that:

1. TF-IDF captures most of the available predictive signal  
2. Minority class difficulty (especially “Safety”) is caused by **low sample size**, not model choice  
3. ANN introduces mild improvements for “Both-impacting” cases  
4. Logistic Regression remains the most interpretable and efficient model  
5. ANN supports the conclusion that **governance impact classification from narrative text is learnable but limited by class imbalance**

This ANN serves as a strong secondary model and helps confirm the robustness of the baseline findings.


***

***

### SVM Step 1 — Imports (LinearSVC + metrics)

In [17]:
# SVM Step 1: Import LinearSVC (linear SVM optimized for text) and metrics for evaluation.
# Note: We use LinearSVC instead of SVC(kernel="linear") because it is much faster and
# designed for high-dimensional sparse data such as TF-IDF.

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

### SVM Step 2 — (Optional) Sanity check shapes of TF-IDF features

In [18]:
# SVM Step 2: Sanity check that the TF-IDF feature matrices and labels are available
# and aligned with the earlier Logistic/ANN experiments.

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)
print("y_train length:", len(y_train))
print("y_test  length:", len(y_test))
print("Classes (encoded):", le.classes_)


X_train shape: (985, 8705)
X_test  shape: (247, 8705)
y_train length: 985
y_test  length: 247
Classes (encoded): ['both' 'neither' 'rights' 'safety']


### SVM Step 3 — Define the LinearSVC model

In [19]:
# SVM Step 3: Define the LinearSVC model.
#
# Key choices:
# - C=1.0: Standard regularization strength; can be tuned if needed.
# - class_weight="balanced": Automatically up-weights minority classes (rights, safety)
#   and down-weights the dominant class (neither).
# - dual=True (default for n_samples > n_features with LinearSVC, but here we let sklearn decide).
#
# LinearSVC does not output probabilities, but is very strong for text classification.

svm_clf = LinearSVC(
    C=1.0,
    class_weight="balanced",
    random_state=42
)

svm_clf

LinearSVC(class_weight='balanced', random_state=42)

### SVM Step 4 — Train the SVM on TF-IDF features

In [20]:
# SVM Step 4: Fit the LinearSVC model using the TF-IDF training features and encoded labels.

svm_clf.fit(X_train, y_train)

LinearSVC(class_weight='balanced', random_state=42)

### SVM Step 5 — Generate predictions on the test set

In [21]:
# SVM Step 5: Use the trained SVM to predict impact type labels on the held-out test set.

y_test_pred_svm = svm_clf.predict(X_test)

### SVM Step 6 — Classification report (precision/recall/F1 per class)

In [22]:
# SVM Step 6: Evaluate the SVM using a detailed classification report,
# aligned with the earlier Logistic and ANN evaluations.

print(classification_report(
    y_test,
    y_test_pred_svm,
    target_names=le.classes_,  # map encoded labels back to ['both', 'neither', 'rights', 'safety']
    digits=3,
    zero_division=0
))


              precision    recall  f1-score   support

        both      0.955     0.840     0.894        25
     neither      0.963     1.000     0.981       210
      rights      0.833     0.556     0.667         9
      safety      0.000     0.000     0.000         3

    accuracy                          0.955       247
   macro avg      0.688     0.599     0.635       247
weighted avg      0.946     0.955     0.949       247



### SVM Step 7 — Confusion matrix

In [23]:
# SVM Step 7: Display the confusion matrix to understand misclassification patterns
# across the four impact-type classes.

cm_svm = confusion_matrix(y_test, y_test_pred_svm)
cm_svm

array([[ 21,   3,   1,   0],
       [  0, 210,   0,   0],
       [  0,   3,   5,   1],
       [  1,   2,   0,   0]])

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### SVM Step 8 — Insights & Key Findings (Final Version)

## ⚖️ SVM (LinearSVC) — Insights & Key Findings

### 1. Overall Performance Is Strong and Consistent With Other Models
The SVM achieved:
- **Accuracy:** 0.955  
- **Macro F1:** 0.635  
- **Macro Recall:** 0.599  

This places the SVM very close to the Logistic Regression (macro F1 = 0.676)  
and slightly below the ANN (macro F1 = 0.668).  
This consistency shows that all three models learn the same underlying text signal.

---

### 2. Excellent Performance on “Both” and “Neither”
The SVM continues the trend seen with Logistic and ANN:
- **Both-impacting:** F1 = 0.894 (very strong)  
- **Neither:** F1 = 0.981 (perfect recall, high precision)

This confirms that the narrative text for these classes carries clear, consistent signals that all models can learn.

---

### 3. “Rights” Class Performance Slightly Lower Than Logistic/ANN
The SVM achieved:
- **Rights:** Precision 0.833, Recall 0.556, F1 0.667

This is slightly below:
- Logistic (F1 = 0.824)  
- ANN (F1 = 0.824)

The confusion matrix shows the model misclassifies some rights-impacting cases as “neither.”  
This suggests that SVM is somewhat more conservative in assigning minority classes.

---

### 4. “Safety” Class Still Not Learned (Expected Dataset Limitation)
As with Logistic and ANN, the SVM achieves:
- **Safety:** Precision = 0.0, Recall = 0.0

This is expected because:
- The dataset contains **only 16 total safety examples**
- Only **3 safety samples** appear in the test set  

No model (linear or nonlinear) can learn a class with such limited representation.  
This is a **data limitation**, not a model failure.

---

### 5. Misclassification Patterns Are Consistent Across All Models
The confusion matrix shows:
- Some “both” predicted as “neither”  
- Some “rights” predicted as “neither”  
- “Safety” mostly predicted as “neither” or “both”

These patterns are almost identical to Logistic and ANN.  
This triangulates the finding that model choices do not meaningfully change the results — the dataset structure drives the performance limits.

---

### 6. SVM Adds Strength to the Model Comparison Story
The SVM confirms:
- Linear text-based models (Logistic, SVM) largely extract the same signal
- ANN does not uncover substantially more nonlinear structure
- Performance on minority classes (especially “rights”) is stable across models
- “Safety” remains challenging due to sample scarcity, not algorithm design

This makes the model comparison section robust and well-justified.

---

### **Summary**
The SVM provides a fast, strong, and stable third model that closely matches performance from Logistic Regression and ANN. Together, the three models triangulate the same conclusion:

- **Impact-type classification from narrative text is highly feasible for “neither”, “both”, and “rights.”**  
- **“Safety” performance is limited by extremely low sample size rather than model capability.**  
- **Across all three models, the overall signal is consistent and reliable**, supporting the strength of the baseline and the broader modeling approach.
